In [1]:
import arxiv
from sentence_transformers import SentenceTransformer
import umap
import nltk
nltk.download('punkt')

c:\Users\marin\anaconda3\envs\clust-arc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\marin\anaconda3\envs\clust-arc\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\marin\anaconda3\envs\clust-arc\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to T

True

In [2]:
import urllib, urllib.request
from bs4 import BeautifulSoup
url = 'https://export.arxiv.org/api/query?search_query=au:robert+koenig&cat:quant-ph&start=0&max_results=59'
with urllib.request.urlopen(url) as response:
    soup = BeautifulSoup(response, "lxml")
    

In [3]:
titles = soup.find_all('title')
titles = list(map(lambda x:x.text, titles))
del titles[0]

titles[-1]

'Locking of accessible information and implications for the security of\n  quantum cryptography'

In [4]:
summaries = soup.find_all('summary')
summaries = list(map(lambda x:x.text, summaries))
summaries[-1]

"  The unconditional security of a quantum key distribution protocol is often\ndefined in terms of the accessible information, that is, the maximum mutual\ninformation between the distributed key S and the outcome of an optimal\nmeasurement on the adversary's (quantum) system. We show that, even if this\nquantity is small, certain parts of the key S might still be completely\ninsecure when S is used in applications, such as for one-time pad encryption.\nThis flaw is due to a locking property of the accessible information: one\nadditional (physical) bit of information might increase the accessible\ninformation by more than one bit.\n"

In [5]:
def retrieve_name(name):
    split_name = name.split(' ', 1)
    return True if split_name[0] == "Robert" or split_name[1] == "Koenig" else False

names = soup.find_all('name')
names = list(map(lambda x:x.text, names))
names = list(filter(retrieve_name, names))
del names[names.index('Robert Howe')]

indices = list([names.index('Alexander Koenig'), names.index('Robert A. Bridges')])
indices

[33, 15]

In [6]:
titles_to_remove = list()
summaries_to_remove = list()

for index in indices:
    titles_to_remove.append(titles[index])
    summaries_to_remove.append(summaries[index])
    
for title, summary in zip(titles_to_remove, summaries_to_remove):
    titles.remove(title)
    summaries.remove(summary)

In [7]:
koenig_papers = list()
for title, summary in zip(titles, summaries):
    koenig_papers.append(title + '.' + '\n\n' + summary)

In [8]:
print(koenig_papers[0])

The conditional entropy power inequality for Gaussian quantum states.

  We propose a generalization of the quantum entropy power inequality involving
conditional entropies. For the special case of Gaussian states, we give a proof
based on perturbation theory for symplectic spectra. We discuss some
implications for entanglement-assisted classical communication over additive
bosonic noise channels.



In [18]:
len(koenig_papers)

57

In [9]:
search = arxiv.Search(query = "quantum", max_results=350, sort_by=arxiv.SortCriterion.SubmittedDate)

In [10]:
papers = list()
for result in search.results():
    papers.append(result.title + '.' + '\n\n' + result.summary)

In [19]:
len(papers)

350

In [11]:
print(papers[0])

Data Encoding For Healthcare Data Democratisation and Information Leakage Prevention.

The lack of data democratization and information leakage from trained models
hinder the development and acceptance of robust deep learning-based healthcare
solutions. This paper argues that irreversible data encoding can provide an
effective solution to achieve data democratization without violating the
privacy constraints imposed on healthcare data and clinical models. An ideal
encoding framework transforms the data into a new space where it is
imperceptible to a manual or computational inspection. However, encoded data
should preserve the semantics of the original data such that deep learning
models can be trained effectively. This paper hypothesizes the characteristics
of the desired encoding framework and then exploits random projections and
random quantum encoding to realize this framework for dense and longitudinal or
time-series data. Experimental evaluation highlights that models trained on
e

In [12]:
#tokenize text into sentences
tokenized_papers = [nltk.sent_tokenize(paper) for paper in papers]

In [20]:
tokenized_papers[0]

['Data Encoding For Healthcare Data Democratisation and Information Leakage Prevention.',
 'The lack of data democratization and information leakage from trained models\nhinder the development and acceptance of robust deep learning-based healthcare\nsolutions.',
 'This paper argues that irreversible data encoding can provide an\neffective solution to achieve data democratization without violating the\nprivacy constraints imposed on healthcare data and clinical models.',
 'An ideal\nencoding framework transforms the data into a new space where it is\nimperceptible to a manual or computational inspection.',
 'However, encoded data\nshould preserve the semantics of the original data such that deep learning\nmodels can be trained effectively.',
 'This paper hypothesizes the characteristics\nof the desired encoding framework and then exploits random projections and\nrandom quantum encoding to realize this framework for dense and longitudinal or\ntime-series data.',
 'Experimental evaluation

In [23]:
papers[0]

'Data Encoding For Healthcare Data Democratisation and Information Leakage Prevention.\n\nThe lack of data democratization and information leakage from trained models\nhinder the development and acceptance of robust deep learning-based healthcare\nsolutions. This paper argues that irreversible data encoding can provide an\neffective solution to achieve data democratization without violating the\nprivacy constraints imposed on healthcare data and clinical models. An ideal\nencoding framework transforms the data into a new space where it is\nimperceptible to a manual or computational inspection. However, encoded data\nshould preserve the semantics of the original data such that deep learning\nmodels can be trained effectively. This paper hypothesizes the characteristics\nof the desired encoding framework and then exploits random projections and\nrandom quantum encoding to realize this framework for dense and longitudinal or\ntime-series data. Experimental evaluation highlights that model

In [24]:
# count number of sentences in each paper
num_sentences = [len(paper) for paper in tokenized_papers]
num_sentences[0]

# remove papers with less than 2 sentences
for i in range(len(papers)):
    if num_sentences[i] <= 3: # 1 title + 2 sentences 
        papers.remove(papers[i])

In [27]:
len(papers)

338

In [28]:
tokenized_papers = [nltk.sent_tokenize(paper) for paper in papers]

In [29]:
tokenized_papers[0]

['Data Encoding For Healthcare Data Democratisation and Information Leakage Prevention.',
 'The lack of data democratization and information leakage from trained models\nhinder the development and acceptance of robust deep learning-based healthcare\nsolutions.',
 'This paper argues that irreversible data encoding can provide an\neffective solution to achieve data democratization without violating the\nprivacy constraints imposed on healthcare data and clinical models.',
 'An ideal\nencoding framework transforms the data into a new space where it is\nimperceptible to a manual or computational inspection.',
 'However, encoded data\nshould preserve the semantics of the original data such that deep learning\nmodels can be trained effectively.',
 'This paper hypothesizes the characteristics\nof the desired encoding framework and then exploits random projections and\nrandom quantum encoding to realize this framework for dense and longitudinal or\ntime-series data.',
 'Experimental evaluation

In [14]:
embeddings = list()
model = SentenceTransformer('all-MiniLM-L6-v2')
for token_paper in tokenized_papers:
    embeddings.append(model.encode(token_paper))

In [15]:
import numpy as np
np.argmin([embedding.shape[0] for embedding in embeddings])
tokenized_papers[335]

['Making Atomic-Level Magnetism Tunable with Light at Room Temperature.',
 'The capacity to manipulate magnetization in two-dimensional dilute magnetic\nsemiconductors (2D-DMSs) using light, specifically in magnetically doped\ntransition metal dichalcogenide (TMD) monolayers (M-doped TX2, where M = V, Fe,\nCr; T = W, Mo; X = S, Se, Te), may lead to innovative applications in\nspintronics, spin-caloritronics, valleytronics, and quantum computation.',
 'This\nPerspective paper explores the mediation of magnetization by light under\nambient conditions in 2D-TMD DMSs and heterostructures.',
 'By combining magneto-LC\nresonance (MLCR) experiments with density functional theory (DFT) calculations,\nwe show that the magnetization can be enhanced using light in V-doped TMD\nmonolayers (e.g., V-WS2, V-WSe2, V-MoS2).',
 'This phenomenon is attributed to\nexcess holes in the conduction and valence bands, as well as carriers trapped\nin magnetic doping states, which together mediate the magnetizat

In [16]:
reducer = umap.UMAP(n_neighbors=2, init='random', n_components=2)
umap_embeddings=list(map(lambda x:reducer.fit_transform(x), embeddings))

c:\Users\marin\anaconda3\envs\clust-arc\lib\site-packages\umap\umap_.py:2344: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
c:\Users\marin\anaconda3\envs\clust-arc\lib\site-packages\umap\umap_.py:132: UserWarning: A large number of your vertices were disconnected from the manifold.
Disconnection_distance = inf has removed 0 edges.
It has fully disconnected 2 vertices.
You might consider using find_disconnected_points() to find and remove these points from your data.
Use umap.utils.disconnected_vertices() to identify them.
  warn(


ValueError: zero-size array to reduction operation maximum which has no identity